In [1]:
load_model = 'vgg19_adamax_30ep_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


from IPython.display import Image
from IPython.display import Markdown

from PIL import Image
import PIL.Image

import pandas as pd

# sklearn metrics
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score


## load the VGG19 based model

In [3]:
model_vgg19 = models.vgg19(pretrained=True)

for param in model_vgg19.parameters():
    param.requires_grad = False

model_vgg19.classifier = nn.Sequential(*list(model_vgg19.classifier.children())[:-1] + [nn.Linear(in_features=4096, out_features=120, bias=True)])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_vgg19 = model_vgg19.to(device)

model_vgg19.load_state_dict(torch.load(load_model), strict=False)
model_vgg19.eval()

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /home/rka73/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [4]:
model = model_vgg19

In [5]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

In [6]:
path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

y_true = []
y_pred = []

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]
            y_true.append(breed)
            y_pred.append(predicted_breed)

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

In [7]:
df = pd.DataFrame(list(zip(y_true, y_pred)),
               columns =['Actual_breed', 'Predicted_breed'])

df

,Actual_breed,Predicted_breed
0,Pomeranian,Pomeranian
1,Pomeranian,Pomeranian
2,Pomeranian,Pomeranian
3,Pomeranian,Pomeranian
4,Pomeranian,Pomeranian
...,...,...
2105,papillon,papillon
2106,papillon,papillon
2107,papillon,papillon
2108,papillon,papillon


## Confusion Matrix

In [8]:
# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))


[[24  0  0 ...  0  0  0]
 [ 0 17  0 ...  0  0  0]
 [ 0  0 20 ...  0  0  0]
 ...
 [ 0  0  0 ... 15  0  0]
 [ 0  0  0 ...  0 14  0]
 [ 0  0  0 ...  0  0 14]]


## Precision, Recall and F1

In [9]:
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

                                precision    recall  f1-score   support

                  Afghan hound      1.000     1.000     1.000        24
           African hunting dog      0.944     1.000     0.971        17
                      Airedale      0.909     0.952     0.930        21
American Staffordshire terrier      0.625     0.882     0.732        17
                   Appenzeller      0.778     0.438     0.560        16
            Australian terrier      0.923     0.600     0.727        20
            Bedlington terrier      1.000     0.947     0.973        19
          Bernese mountain dog      0.846     1.000     0.917        22
              Blenheim spaniel      0.947     0.947     0.947        19
                 Border collie      0.727     0.533     0.615        15
                Border terrier      0.857     1.000     0.923        18
                   Boston bull      0.850     0.895     0.872        19
          Bouvier des Flandres      0.765     0.867     0.812  

## Macro F1 Score

In [10]:
from sklearn.metrics import f1_score

f1_score(y_true, y_pred, average='macro')


0.8509589248736255

## Micro F1 Score

In [11]:
f1_score(y_true, y_pred, average='micro')


0.8559241706161137

## Weighted F1 Score

In [12]:
f1_score(y_true, y_pred, average='weighted')

0.8537057858287432

## Cohen Kappa Score

In [13]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_true, y_pred)

0.854687277706241

## Total Accuracy calculated from per class accuracy

In [14]:
# calculated from per class accuracy
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

85.17253524625815

## Per Breed Accuracy

### Top 10 Breeds with least accuracy

In [15]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 26.666666666666668),
 ('Appenzeller', 43.75),
 ('Staffordshire bullterrier', 50.0),
 ('Walker hound', 50.0),
 ('Siberian husky', 50.0),
 ('Border collie', 53.333333333333336),
 ('miniature poodle', 56.25),
 ('standard schnauzer', 56.25),
 ('collie', 56.25),
 ('toy poodle', 56.25)]

### Top 10 Breeds with most accuracy

In [16]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Japanese spaniel', 100.0),
 ('Brabancon griffon', 100.0),
 ('Shih Tzu', 100.0),
 ('Irish setter', 100.0),
 ('clumber', 100.0),
 ('schipperke', 100.0),
 ('chow', 100.0),
 ('komondor', 100.0),
 ('Old English sheepdog', 100.0),
 ('Chihuahua', 100.0)]